In [1]:
# third-party
import pandas as pd
import scipy
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# local
from get_data import transform_overview_on_target, transform_overview_on_overall
from performance import get_breath_parameters_performance
from delay_performance import normality_test
from performance import bland_altman_plot, get_relative_errors
from get_data import get_data_by_id_activity
from constants import CATEGORICAL_PALETTE


In [2]:
with open('Results/results.pickle', 'rb') as file:
    overview = pickle.load(file)

overview_middle = {}
for id in overview.keys():
    overview_middle[id] = {key: value for key, value in overview[id].items() if key in ['SNBm', 'UALm', 'UARm']}

# remove middle activities
for id in overview.keys():
    del overview[id]["SNBm"]
    del overview[id]["UALm"]
    del overview[id]["UARm"]

In [3]:
acquisition_folderpath = 'Aquisicao'
data, _ = get_data_by_id_activity(acquisition_folderpath)


Getting data for participants...
 --------- 7OYX ---------------
 --------- NO15 ---------------
 --------- G8B7 ---------------
 --------- EPE2 ---------------
 --------- HAK8 ---------------
 --------- 1BST ---------------
 --------- 83J1 ---------------
 --------- QMQ7 ---------------
 --------- 9TUL ---------------
 --------- FTD7 ---------------
 --------- Y6O3 ---------------
 --------- 2QWT ---------------
 --------- F9AF ---------------
 --------- P4W9 ---------------
 --------- W8Z9 ---------------
 --------- D4GQ ---------------


In [4]:
def plot_resp_param(data, overview, id, activity, sensor, sampling_freq=100):

    sensor_dict = {"MAG": "mag", "PZT": "pzt"}

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # plot sensor and airflow signals
    fig.add_trace(go.Scatter(x=np.linspace(1, len(data[id][activity][sensor_dict[sensor]]),len(data[id][activity][sensor_dict[sensor]]))/sampling_freq, 
                             y=data[id][activity][sensor_dict[sensor]], 
                             name=sensor))
    fig.add_trace(go.Scatter(x=np.linspace(1, len(data[id][activity]['airflow']),len(data[id][activity]['airflow']))/sampling_freq,
                             y=data[id][activity]['airflow'], 
                            name="Airflow", line={"color":"white"}), secondary_y=True)

    # add FR events
    peak_valley = {"TP_i": "valleys", "TP_e": "peaks", "FN_i": "valleys", "FN_e": "peaks", "FP_i": "valleys", "FP_e": "peaks"}

    for event in ["TP_i", "TP_e"]:
        fig.add_trace(go.Scatter(x=np.array(overview[id][activity][sensor][event])/sampling_freq, y=data[id][activity][sensor_dict[sensor]][overview[id][activity][sensor][event]],
                            mode="markers",
                            name=f"TP {peak_valley[event]}"))
        
        fig.add_trace(go.Scatter(x=np.array(overview[id][activity]["Airflow"][peak_valley[event]])/sampling_freq, y=data[id][activity]['airflow'][overview[id][activity]["Airflow"][peak_valley[event]]],
                        mode="markers",
                        name=f"ref {peak_valley[event]}", marker={"color":"white"}), secondary_y=True)
    
    for event in ["FN_i", "FN_e"]:
        fig.add_trace(go.Scatter(x=np.array(overview[id][activity][sensor][event])/sampling_freq, y=data[id][activity][sensor_dict[sensor]][overview[id][activity][sensor][event]],
                            mode="markers",
                            name=f"FN {peak_valley[event]}"))
        
    for event in ["FP_i", "FP_e"]:
        fig.add_trace(go.Scatter(x=np.array(overview[id][activity][sensor][event])/sampling_freq, y=data[id][activity][sensor_dict[sensor]][overview[id][activity][sensor][event]],
                            mode="markers",
                            name=f"FP {peak_valley[event]}"))


    TP_peaks_tuple = [(event, True, "peak") for event in overview[id][activity][sensor]["TP_e"]]
    TP_valleys_tuple = [(event, True, "valley") for event in overview[id][activity][sensor]["TP_i"]]
    FN_peaks_tuple = [(event, False, "peak") for event in overview[id][activity][sensor]["FN_e"]]
    FN_valleys_tuple = [(event, False, "valley") for event in overview[id][activity][sensor]["FN_i"]]
    extrema = sorted(TP_peaks_tuple + TP_valleys_tuple + FN_peaks_tuple + FN_valleys_tuple)

    
    extrema_pairs_peaks = np.asarray([
        (extrema[i][0], extrema[i+1][0],
            all((extrema[i][1], extrema[i+1][1])))
        for i in range(0, len(extrema)-1)
        if extrema[i][2] == "peak" and extrema[i+1][2] == "valley"
    ])

    extrema_pairs_valleys = np.asarray([
        (extrema[i][0], extrema[i+1][0],
            all((extrema[i][1], extrema[i+1][1])))
        for i in range(0, len(extrema)-1)
        if extrema[i][2] == "valley" and extrema[i+1][2] == "peak"
    ])

    extrema_pairs_peaks = extrema_pairs_peaks[extrema_pairs_peaks[:,-1] != 0]
    extrema_pairs_valleys = extrema_pairs_valleys[extrema_pairs_valleys[:,-1] != 0]

    for i,ind in enumerate(extrema_pairs_peaks):
        try:
            exp_duration = overview[id][activity][sensor]["tE (s)"][i]
            fig.add_vrect(x0=ind[0]/sampling_freq, x1=ind[0]/sampling_freq+exp_duration, fillcolor=CATEGORICAL_PALETTE[0], 
                            opacity=0.2, line_width=0.2, line_color='white', annotation_text="exp", annotation_position="top left",)
        except Exception as e:
            print(e)

    for i, ind in enumerate(extrema_pairs_valleys):  
        try:
            insp_duration = overview[id][activity][sensor]["tI (s)"][i]
            fig.add_vrect(x0=ind[0]/sampling_freq, x1=ind[0]/sampling_freq+insp_duration, fillcolor=CATEGORICAL_PALETTE[1], 
                            opacity=0.2, line_width=0.2, line_color='white', annotation_text="insp", annotation_position="bottom left",)
        except Exception as e:
            print(e)
                
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(showgrid=False)
    fig.show()

    print(f'sensor: {overview[id][activity][sensor]["tI (s)"]} | airflow: {overview[id][activity][sensor]["tI airflow (s)"]}')
    

In [5]:

plot_resp_param(data, overview, id="EPE2", activity="SGB", sensor="MAG")


sensor: [4.97 5.54 3.83 6.19] | airflow: [4.74 4.03 3.91 5.55]


In [18]:
plot_resp_param(data, overview, id="2QWT", activity="MIXB", sensor="MAG")

sensor: [3.09 6.55 6.36 5.81 4.54 3.66 3.26 4.2  6.1  5.6  4.52 3.43] | airflow: [2.92 4.26 4.67 4.27 4.29 3.64 3.   4.18 4.46 3.98 3.35 3.07]


### Activity-specific only RP analysis

In [7]:
# Transform overview from participant specific into all participants

overview_all_participants = transform_overview_on_target(overview, target="Activity")
overview_all_participants.keys()

dict_keys(['SNB', 'SGB', 'MIXB', 'STNB', 'MCH', 'SQT', 'AAL', 'AAR', 'ALL', 'ALR', 'UAL', 'UAR', 'SE', 'SS', 'TR'])

In [8]:
performance_all_participants = get_breath_parameters_performance(overview_all_participants, target="Activity")
performance_all_participants

,Activity,Sensor,MAE Ti (s),MRE Ti (%),MAE Te (s),MRE Te (%),MAE Tb (s),MRE Tb (%)
0,SNB,MAG,0.30 $\pm$ 0.46,14.09 $\pm$ 19.58,0.30 $\pm$ 0.45,11.36 $\pm$ 14.43,0.20 $\pm$ 0.33,4.31 $\pm$ 6.95
1,SNB,PZT,0.55 $\pm$ 0.58,27.64 $\pm$ 24.98,0.56 $\pm$ 0.57,21.56 $\pm$ 16.98,0.43 $\pm$ 0.55,9.97 $\pm$ 12.22
2,SGB,MAG,1.08 $\pm$ 0.88,25.85 $\pm$ 22.45,1.15 $\pm$ 0.84,21.17 $\pm$ 18.40,0.81 $\pm$ 0.70,8.05 $\pm$ 7.09
3,SGB,PZT,1.38 $\pm$ 1.03,32.48 $\pm$ 22.36,1.65 $\pm$ 1.29,28.98 $\pm$ 23.18,1.10 $\pm$ 1.26,10.99 $\pm$ 12.98
4,MIXB,MAG,0.35 $\pm$ 0.44,14.33 $\pm$ 16.50,0.35 $\pm$ 0.46,11.69 $\pm$ 13.70,0.26 $\pm$ 0.39,4.51 $\pm$ 5.99
5,MIXB,PZT,0.73 $\pm$ 0.67,29.56 $\pm$ 22.87,0.79 $\pm$ 0.92,24.93 $\pm$ 20.09,0.53 $\pm$ 0.79,8.96 $\pm$ 10.81
6,STNB,MAG,0.27 $\pm$ 0.40,12.29 $\pm$ 16.54,0.30 $\pm$ 0.44,11.28 $\pm$ 15.81,0.23 $\pm$ 0.47,4.77 $\pm$ 8.49
7,STNB,PZT,0.55 $\pm$ 0.49,25.93 $\pm$ 19.77,0.59 $\pm$ 0.55,23.01 $\pm$ 19.18,0.33 $\pm$ 0.42,7.30 $\pm$ 9.68
8,MCH,MAG,0.39 $\pm$ 0.38,20.66 $\pm$ 16.49,0.36 $\pm$ 0.34,19.19 $\pm$ 17.64,0.39 $\pm$ 0.47,9.70 $\pm$ 9.95
9,MCH,PZT,0.63 $\pm$ 0.51,31.46 $\pm$ 22.52,0.59 $\pm$ 0.53,30.60 $\pm$ 24.42,0.58 $\pm$ 0.54,15.08 $\pm$ 14.78


### Overall RP analysis

In [9]:
# Transform overview from participant specific into all participants
overview_all = transform_overview_on_overall(overview)
overview_all.keys()

dict_keys(['MAG', 'Airflow', 'PZT'])

In [10]:
get_breath_parameters_performance(overview_all, target=None)

N=3357 tI for MAG
N=3275 tI for PZT
N=3351 tE for MAG
N=3261 tE for PZT
N=3263 tB for MAG
N=3185 tB for PZT


,Parameter,Sensor,Abs. error (s),Rel. error (%),Slope,Intercept,R^2
0,tI,MAG,0.38 $\pm$ 0.42,19.33 $\pm$ 18.79,1.02,0.01,0.65
1,tI,PZT,0.61 $\pm$ 0.56,31.18 $\pm$ 24.29,0.88,0.24,0.40
2,tE,MAG,0.39 $\pm$ 0.43,18.20 $\pm$ 17.72,0.87,0.23,0.71
3,tE,PZT,0.63 $\pm$ 0.61,29.29 $\pm$ 23.32,0.96,0.05,0.55
4,tB,MAG,0.31 $\pm$ 0.41,7.40 $\pm$ 9.20,0.98,0.06,0.91
5,tB,PZT,0.52 $\pm$ 0.62,12.57 $\pm$ 13.76,0.98,0.08,0.80


In [11]:
import plotly.graph_objects as go

for metric in ["tI", "tE", "tB"]:
    rel_error_mag = get_relative_errors(overview_all["MAG"][f"{metric} airflow (s)"], overview_all["MAG"][f"{metric} (s)"])
    rel_error_pzt = get_relative_errors(overview_all["PZT"][f"{metric} airflow (s)"], overview_all["PZT"][f"{metric} (s)"])

    print(f"{metric}: {scipy.stats.mannwhitneyu(rel_error_mag, rel_error_pzt)}")

    fig = go.Figure()
    fig.add_trace(go.Violin(y=rel_error_mag, name="MAG"))
    fig.add_trace(go.Violin(y=rel_error_pzt, name="PZT"))

    # fig.show()

    # normality_test(rel_error_mag, sensor="MAG", type=f"relative error {metric}")
    # normality_test(rel_error_pzt, sensor="PZT", type=f"relative error {metric}")


tI: MannwhitneyuResult(statistic=5289814.5, pvalue=0.0078404340833656)
tE: MannwhitneyuResult(statistic=5372566.5, pvalue=0.23970341962448738)
tB: MannwhitneyuResult(statistic=5118386.0, pvalue=0.2969885432510643)


### Bias and variability analysis by activity

In [16]:
for id in overview.keys():
    for activity in overview[id].keys():
        for sensor in ["MAG", "PZT"]:
            if any(overview[id][activity][sensor]["tE (s)"] > 10):
                print(f'{overview[id][activity][sensor]["tE (s)"]} ({id} - {activity} - {sensor})')

[ 6.79  3.08  6.21 11.4   4.14  6.32] (QMQ7 - SGB - PZT)
[10.76  7.98  5.14] (Y6O3 - SGB - PZT)
[ 1.53  1.38  5.13 10.36  3.05  3.25  2.84  1.8 ] (2QWT - MIXB - PZT)


In [13]:
from performance import get_bias_variability

get_bias_variability(overview_all_participants, target="Activity", metric="tI", relative_error=False)

,Activity,Sensor,Bias (s),Variability (s)
0,SNB,MAG,0.126,0.535
1,SNB,PZT,0.283,0.754
2,SGB,MAG,0.366,1.351
3,SGB,PZT,-0.393,1.684
4,MIXB,MAG,0.181,0.529
5,MIXB,PZT,0.236,0.968
6,STNB,MAG,0.040,0.486
7,STNB,PZT,0.311,0.667
8,MCH,MAG,0.020,0.544
9,MCH,PZT,-0.090,0.808


In [14]:
activity = "UAL"
for metric in ["tI", "tE", "tB"]:
        bland_altman_plot(overview_all["MAG"][f"{metric} (s)"], overview_all["MAG"][f"{metric} airflow (s)"], overview_all["PZT"][f"{metric} (s)"], overview_all["PZT"][f"{metric} airflow (s)"], metric)
        #bland_altman_plot(overview_all_participants[activity]["MAG"][f"{metric} (s)"], overview_all_participants[activity]["MAG"][f"{metric} airflow (s)"], overview_all_participants[activity]["PZT"][f"{metric} (s)"], overview_all_participants[activity]["PZT"][f"{metric} airflow (s)"], metric, activity)


MAG | mean: 0.058, lloa: -1.05, uloa: 1.17 
PZT | mean: 0.014, lloa: -1.60, uloa: 1.63 


MAG | mean: -0.064, lloa: -1.18, uloa: 1.06 
PZT | mean: -0.035, lloa: -1.75, uloa: 1.68 


MAG | mean: -0.004, lloa: -1.02, uloa: 1.01 
PZT | mean: -0.021, lloa: -1.61, uloa: 1.57 
